In [17]:
from unsupervised_topic_segmentation import core, eval, types, dataset
import create_test_data
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ast

In [3]:
test = pd.read_parquet('data/ami_embed_results/embed.snappy.parquet')

In [18]:
test['embeddings'] = np.empty((len(test), 0)).tolist()
for i in range(len(test)):
    test['embeddings'][i] = ast.literal_eval(test['finished_embeddings_str'][i])[0]
test.drop(columns=['finished_embeddings_str'], inplace=True)

In [ ]:
new_algorithm = types.BERTSegmentation(
    sentence_comparison_window=50,
    text_tiling=types.NewSegmentation(
        stdevs=1))

new_algorithm